In [1]:
!pip3 install pandas
!pip3 install xgboost
!pip3 install sklearn
!pip3 install six==1.10
!pip3 install matplotlib
!pip3 install statsmodels
!pip3 install keras==2.2
!pip3 install tensorflow
!pip3 install np_utils

In [2]:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import xgboost as xgb
import math

from xgboost.sklearn import XGBClassifier
from xgboost.sklearn import XGBRegressor
from xgboost import plot_tree

#from graphviz import Digraph

from sklearn import cross_validation, metrics
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from math import sqrt

%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


# Load Data

In [3]:
kyoto_1_score = pd.read_csv('dataset/data1_score.csv')
kyoto_1_score.head(5)

,userid,score
0,ds101,80
1,ds102,80
2,ds103,60
3,ds104,50
4,ds105,100


In [4]:
kyoto_1_all = pd.read_csv('dataset/dataset1_features.csv')
kyoto_1_p1 = pd.read_csv('dataset/total01_1213.csv')
kyoto_1_p2 = pd.read_csv('dataset/total01_0123~0124.csv')
kyoto_1_p3 = pd.read_csv('dataset/total01_1122.csv')
kyoto_week = pd.read_csv('dataset/Week1filter.csv')

# Merge Datasets

In [5]:
kyoto_x_all = pd.merge(kyoto_1_all, kyoto_1_score, on='userid')
kyoto_x_1 = pd.merge(kyoto_1_p1, kyoto_1_score, on='userid')
kyoto_x_2 = pd.merge(kyoto_1_p2, kyoto_1_score, on='userid')
kyoto_x_3 = pd.merge(kyoto_1_p3, kyoto_1_score, on='userid')
kyoto_week = pd.merge(kyoto_1_p3, kyoto_1_score, on='userid')

In [6]:
kyoto_x_p = kyoto_x_1.append([kyoto_x_2, kyoto_x_3])
kyoto_x_p.rename(index=str, columns={"ADD BOOKMARK": "Add_BookmarkC"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"CLOSE": "closec"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"MARKERC": "markerc"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"MEMOC": "memoc"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"OPEN": "openc"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"PAGE_JUMP": "Page_JumpC"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"mobile": "mobilec"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"pc": "pcc"}, inplace=True)
kyoto_x_p.rename(index=str, columns={"tablet": "tabletc"}, inplace=True)
set(kyoto_x_p) - set(kyoto_x_all)

set()

In [7]:
kyoto_1 = kyoto_x_all
#kyoto_1 = kyoto_x_p
#kyoto_1 = kyoto_week
#kyoto_1 = pd.concat([kyoto_x_all, kyoto_x_p])
kyoto_1.fillna(0.0, inplace=True)
kyoto_1 = kyoto_1.drop(['userid'], axis=1)

#list(kyoto_1)[11]

kyoto_1 = kyoto_1.drop([list(kyoto_1)[14]], axis=1)

# Preprocess

In [8]:

outlier = 3
#kyoto_1 = kyoto_1[np.abs(kyoto_1.pcc-kyoto_1.pcc.mean()) <= (outlier * kyoto_1.pcc.std())]
#kyoto_1 = kyoto_1[np.abs(kyoto_1.tabletc-kyoto_1.tabletc.mean()) <= (outlier * kyoto_1.tabletc.std())]
#kyoto_1 = kyoto_1 * 10
#kyoto_1 = np.log(kyoto_1)
#kyoto_1.fillna(0.0, inplace=True)
#kyoto_1 = (kyoto_1 - kyoto_1.mean()) / (kyoto_1.max() - kyoto_1.min()) * 10
#kyoto_1.fillna(0.0, inplace=True)

kyoto_1
#kyoto_1.plot.box(sym='r+')

,bookmarkc,closec,markerc,memoc,mobilec,openc,pcc,tabletc,watchc,SEARCH,...,Page_JumpC,Add_BookmarkC,Delete_BookmarkC,Add_MemoC,Delete_MemoC,Change_MemoC,Add_MarkerC,Delete_MarkerC,Readpages,score
0,0,3,0,0,0,27,1240,66,1306,0,...,65,0,0,0,0,0,0,0,1211,80
1,0,0,0,0,0,1,8,0,8,0,...,0,0,0,0,0,0,0,0,7,80
2,2,1,2,0,45,5,409,0,454,0,...,1,1,1,0,0,0,2,0,443,60
3,2,0,2,2,0,4,89,0,89,0,...,2,1,1,1,0,0,1,1,77,50
4,0,3,0,0,0,5,540,0,540,0,...,2,0,0,0,0,0,0,0,530,100
5,0,0,0,0,0,2,267,0,267,9,...,2,0,0,0,0,0,0,0,253,85
6,0,3,0,0,379,6,239,0,618,0,...,0,0,0,0,0,0,0,0,609,90
7,0,0,0,0,0,8,0,1950,1950,0,...,0,0,0,0,0,0,0,0,1942,100
8,0,3,0,0,0,10,362,0,362,0,...,4,0,0,0,0,0,0,0,344,60
9,0,6,0,0,0,10,863,0,863,0,...,2,0,0,0,0,0,0,0,845,75


In [9]:

kyoto_y = kyoto_1['score']
kyoto_x = kyoto_1.drop(['score'], axis=1)

# Train Model

In [10]:
xgb2 = XGBRegressor(n_estimators=10000, 
                    learning_rate=0.01, 
                    gamma=0, 
                    subsample=0.9,
                    colsample_bytree=0.9, 
                    max_depth=5)

In [11]:
mlp = MLPRegressor(hidden_layer_sizes=(5,),
                   activation='relu',
                   solver='adam',
                   learning_rate='adaptive',
                   max_iter=10000,
                   learning_rate_init=0.01,
                   alpha=0.01)

In [12]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(25, input_dim=25, activation='relu'))
    model.add(Dense(8, input_dim=8, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
mse_xgb_list = []
mse_pcx_list = []
mse_ols_list = []
mse_svr_list = []
mse_mlp_list = []
mse_pcr_list = []
mse_regr_list = []
mse_knn_list = []

mse_xgbt_list = []

for i in range(0, 50):
    
    print (i)
    
    X_train, X_test, y_train, y_test = train_test_split(kyoto_x, kyoto_y, test_size=0.1)
    scale = StandardScaler()
    X_train = scale.fit_transform(X_train)
    X_test = scale.fit_transform(X_test)
    
    # XGBoost
    xgb2.fit(X_train, y_train)
    xgr_train_y = xgb2.predict(X_train)
    xgr_pred_y = xgb2.predict(X_test)
    mse_xgb_list.append(sqrt(mean_squared_error(y_test, xgr_pred_y)))
    mse_xgbt_list.append(sqrt(mean_squared_error(y_train, xgr_train_y)))
    
    # PCX
    pca = PCA(n_components=16)# adjust yourself
    pca.fit(X_train)
    X_pca_train = pca.transform(X_train)
    X_pca_test = pca.transform(X_test)    
    xgb2.fit(X_pca_train, y_train)
    pcx_pred_y = xgb2.predict(X_pca_test)
    mse_pcx_list.append(sqrt(mean_squared_error(y_test, pcx_pred_y)))
    
    # Random Forest Regressor
    regr = RandomForestRegressor(max_depth=5, random_state=0)
    regr.fit(X_train, y_train)
    regr_pred_y = regr.predict(X_test)
    mse_regr_list.append(sqrt(mean_squared_error(y_test, regr_pred_y)))
    
    # linear regression
    ols_model = sm.OLS(endog=y_train, exog=X_train, missing='drop')
    ols_model = ols_model.fit()
    ols_pred_y = ols_model.predict(exog=X_test)
    mse_ols_list.append(sqrt(mean_squared_error(y_test, ols_pred_y)))
    
    # support vector regression
    clf = SVR(C=1.0, epsilon=0.2)
    svr_model = clf.fit(X_train, y_train) 
    svr_pred_y = svr_model.predict(X_test)
    mse_svr_list.append(sqrt(mean_squared_error(y_test, svr_pred_y)))
    
    # Multi-layer Perceptron regressor
    mlp_model = mlp.fit(X_train, y_train)
    mlp_pred_y = mlp_model.predict(X_test)
    mse_mlp_list.append(sqrt(mean_squared_error(y_test, mlp_pred_y)))
    
    # Princple Componment Regression
    pca = PCA(n_components=16)# adjust yourself
    pca.fit(X_train)
    X_pca_train = pca.transform(X_train)
    X_pca_test = pca.transform(X_test)
    pcr_model = sm.OLS(endog=y_train, exog=X_pca_train, missing='drop')
    pcr_model = pcr_model.fit()
    pcr_pred_y = pcr_model.predict(exog=X_pca_test)
    mse_pcr_list.append(sqrt(mean_squared_error(y_test, pcr_pred_y)))
    
    # keras NN
    knn = KerasRegressor(build_fn=baseline_model, epochs=5000, batch_size=5, verbose=0)
    knn.fit(X_train, y_train)
    knn_pred_y = knn.predict(X_test)
    mse_knn_list.append(sqrt(mean_squared_error(y_test, knn_pred_y)))



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


# Boxplot

In [ ]:
matplotlib.rcParams.update({'font.size': 18})

fig1, ax1 = plt.subplots(figsize=(18, 8))
ax1.set_title('RMSE')
box_data = [mse_xgb_list,
            mse_xgbt_list,
            mse_pcx_list, 
            mse_regr_list,
            mse_svr_list, 
            mse_mlp_list, 
            mse_pcr_list]

#ax1.set_yscale('log')
ax1.boxplot(box_data)
ax1.set_ylim([-100, 800])
ax1.set_xticklabels(['XGBoost',
                     'XGBoost(Train Loss)',
                     'PCXGBoox', 
                     'RFR', 
                     'SVR', 
                     'ANN', 
                     'PCR',
                     'Keras NN'], rotation=45)

In [ ]:
print ('XGBoost:', sum(mse_xgb_list) / len(mse_xgb_list))
print ('XGBoost(train_loss):', sum(mse_xgbt_list) / len(mse_xgbt_list))
print ('PCXGBoost:', sum(mse_pcx_list) / len(mse_pcx_list))
print ('Random Forest Regression:', sum(mse_regr_list) / len(mse_regr_list))
print ('Support Vector Regression:', sum(mse_svr_list) / len(mse_svr_list))
print ('Artificial Neural Network:', sum(mse_mlp_list) / len(mse_mlp_list))
print ('Principle Component Regression:', sum(mse_pcr_list) / len(mse_pcr_list))
print ('Keras Neural Network:', sum(mse_pcr_list) / len(mse_pcr_list))